In [ ]:
# 107303538_1028_hw7_1

In [ ]:
import torch
import torch.nn as nn
import numpy as np

載入所有資訊成bikes並處理資料

In [ ]:
# 處理資料
bikes_numpy = np.loadtxt( 
  "hour-fixed.csv",
  dtype=np.float32, # 轉 32-float
  delimiter=",", # 分隔號
  skiprows=1, # 去除第一列的文字
  # 對於某性質進行轉換
  converters={1: lambda x: float(x[8:10])}) # 把日期(第一column)的數據抓給x x中的8和9(也就是日期部分)(2011/01/01) 轉成float
  # lambda 為匿名函數 可一行撰寫 :後面為指令 :前面為要被下指令的變數
bikes = torch.from_numpy(bikes_numpy) # 轉成tensor
bikes 

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

bikes[17520, 17]，[天數*小時數，參數]

In [ ]:
bikes.shape, bikes.stride() # [列,行] (17個換一列, 1個換一行(為continue)) [天數*小時數,參數]

(torch.Size([17520, 17]), (17, 1))

In [ ]:
""" bikes [天數*小時數,參數]

          編號 日期 季節 年 月 小時.....  
  第一天第一個小時
  第一天第二個小時
  :
  :
"""

' bikes [天數*小時數,參數]\n\n          編號 日期 季節 年 月 小時.....  \n  第一天第一個小時\n  第一天第二個小時\n  :\n  :\n'

變三維資料daily_bikes[730, 24, 17]，[天數，小時數，參數]

In [ ]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1]) # -1為一個placeholder("-1" 的意思為自動計算維度) 每24小時組合成一個列資料 相當於17(由上面原始矩陣得知)
# view是把tensor變成一維資料 後再依指定方式重新排列
daily_bikes.shape, daily_bikes.stride() # [縱,列,行] [page][row][col] (17*24換一面，17換一列，1換一行)

(torch.Size([730, 24, 17]), (408, 17, 1))

In [ ]:
"""  x  編號 日期 季節 年 月 小時.....   z   第一天 第二天.....
   y                       y
  第一小時                    第一小時
  第二小時                    第二小時
  :                        :
  :                        :  
"""

'  x  編號 日期 季節 年 月 小時.....   z   第一天 第二天.....\n   y                       y\n  第一小時                    第一小時\n  第二小時                    第二小時\n  :                        :\n  :                        :  \n'

三維資料轉置列與行，變三維資料daily_bikes[730, 17, 24]，[天數，參數，小時數]

In [ ]:
daily_bikes = daily_bikes.transpose(1, 2) # 轉置列(1)和行(2)
daily_bikes.shape, daily_bikes.stride() # [天數,參數,小時數]

(torch.Size([730, 17, 24]), (408, 1, 17))

In [ ]:
""" daily_bikes [天數,參數,小時數]

   x  第一小時 第二小時 .....   z  第一天 第二天.....
   y                  y
   編號                編號  
   日期                日期 
   季節                季節 
   年                 年
   月                 月
   小時                小時       
   :                  :
   :                  :
"""

' daily_bikes [天數,參數,小時數]\n\n   x  第一小時 第二小時 .....   z  第一天 第二天.....\n   y                  y\n   編號                編號  \n   日期                日期 \n   季節                季節 \n   年                 年\n   月                 月\n   小時                小時       \n   :                  :\n   :                  :\n'

(1. 一天24小時)只處理第一天24小時的weather，並建立onehot整理weather(1-4)

In [ ]:
first_day = bikes[:24].long() # 取第一天的資料(0-23小時) 所有17個參數 bikes[:24] = bikes[:24,]
first_day.shape # [24,17]
weather_onehot = torch.zeros(first_day.shape[0], 4) # 建立一個(24,4)空矩陣 會需要4是因為天氣資料為1-4
first_day[:,9] # 取第一天 不管甚麼時候的weather情況(weather為第9格)

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

整理weather成onehot

In [ ]:
weather_onehot.scatter_(
  dim=1, # 跳列(內建3D)
  index=first_day[:,9].unsqueeze(1).long() - 1, # unsqueeze(1)永遠為1 將原本的1-4轉到0-3 
  value=1.0) # 轉換值1.
# scatter (dim指定跳行或列 index為變化的索引 value為變化的值)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

取第一天第一個小時的所有資料

In [ ]:
torch.cat((bikes[:24], weather_onehot), 1)[:1] # bikes[:24,]為(24,17) weather_onehot為(24,4) 行合併為(24,21) 最後[:1,]又只取第一行(0,21)
# torch.cat((A,B),dim) 指定某dim(0為列 1為行) 把AB合併起來 

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

(2. 每天24小時)處理全部天數每天24小時的weather

In [ ]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2]) # 空torch[天數,4,小時數]
daily_weather_onehot.shape

torch.Size([730, 4, 24])

整理weather成onehot

In [ ]:
daily_weather_onehot.scatter_(
  1, # dim變化列
  daily_bikes[:,9,:].long().unsqueeze(1) - 1, # index取第9個weather
  1.0) # 指定為1
daily_weather_onehot.shape # 總共會有24*730個1

torch.Size([730, 4, 24])

取全部天數每天24小時的(全部資料+onehot)

In [ ]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1) 
# 擴充 daily_bikes(730, 17, 24) daily_weather_onehot(730, 4, 24) 變成(730, 21, 24)

其他正規化或標準化處理方法

In [ ]:
# 處理weather(第9個) 正規化
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0  # [730天，weather資料，24小時]

正規化

In [ ]:
# 處理tmeperature(第10個) 正規化
temp = daily_bikes[:, 10, :] # [天數,參數,小時數]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min) / (temp_max - temp_min)) # 公式

標準化

In [ ]:
# 處理tmeperature(第10個) 標準化
temp = daily_bikes[:, 10, :] 
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp)) / torch.std(temp)) # 公式